In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import resnet18, ResNet50_Weights
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import pandas as pd


In [3]:
image_dir = "Images/"
train_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/train_1.txt"
test_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/test_1.txt"


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
])

In [5]:
def load_data(txt_file, image_dir):
    data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
    image_paths = [image_dir + img_name for img_name in data[0]]
    scores = data[1].values
    return image_paths, scores

def create_dataloader(image_paths, scores, batch_size=64):
    images = []
    for img_path in image_paths:
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        images.append(image)

    images = torch.stack(images)
    scores = torch.tensor(scores, dtype=torch.float32)

    dataset = TensorDataset(images, scores)


    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [6]:
train_image_paths, train_scores = load_data(train_txt, image_dir)
test_image_paths, test_scores = load_data(test_txt, image_dir)

train_loader = create_dataloader(train_image_paths, train_scores)
test_loader = create_dataloader(test_image_paths, test_scores)

/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_37702/3863767359.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_37702/3863767359.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(txt_file, delim_whitespace=True, header=None)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=True)
# model = torch.nn.DataParallel(model)

# # Set the model to run on the device
model.load_state_dict(torch.load("resnet18_finetuned.pth"))
model.eval()
# model = model.to(device)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_37702/30654741.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute ar

RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([1, 512]) from checkpoint, the shape in current model is torch.Size([1000, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([1000]).

In [44]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [45]:
num_epochs = 10  
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    progress = 0
    for inputs, targets in train_loader:
        progress += 1
        print(f'Progress: {progress}/{len(train_loader)}', end='\r')
        optimizer.zero_grad()
        # inputs = inputs.to(device)
        # targets = targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    

model.eval()
running_loss = 0.0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        running_loss += loss.item() * inputs.size(0)

test_loss = running_loss / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')



Epoch 1/10, Loss: 0.5618
Epoch 2/10, Loss: 0.1152
Epoch 3/10, Loss: 0.0748
Epoch 4/10, Loss: 0.0596
Epoch 5/10, Loss: 0.0568
Epoch 6/10, Loss: 0.0429
Epoch 7/10, Loss: 0.0345
Epoch 8/10, Loss: 0.0347
Epoch 9/10, Loss: 0.0288
Epoch 10/10, Loss: 0.0259
Test Loss: 0.0859


In [46]:
torch.save(model.state_dict(), 'resnet18_finetuned.pth')


In [1]:
def PrintBeauty(img_path):
    image = Image.open(img_path).convert('RGB')
    image = transform(image)
    print(model(image.unsqueeze(0)).data[0][0])

PrintBeauty("")

NameError: name 'Image' is not defined